In [ ]:
%matplotlib inline
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import dask.array as da
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
import cmocean.cm as cmo
import h5py
from datetime import datetime as dt
from scipy.interpolate import interp1d

plt.style.use('ggplot')

In [ ]:
dataurl = 'http://oceanmodeling.pmc.ucsc.edu:8080/thredds/dodsC/ccsra_2016a_phys_agg_zlevs/fmrc/CCSRA_2016a_Phys_ROMS_z-level_(depth)_Aggregation_best.ncd'

In [ ]:
### Open the dataset with xarray
ds = xr.open_dataset(dataurl)

In [ ]:
### Let's see what's in the dataset
# hooo boy, there's a lot here with all the metadata and everything
ds

In [ ]:
### making a timenum column for plotting
ds['mtime'] = mdates.date2num(ds.time)

In [ ]:
### Import modules for mapping
import cartopy.crs as ccrs
import cartopy
import matplotlib.gridspec as gridspec # GRIDSPEC !
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

In [ ]:
time_index = 1500
var = ds.temp[time_index,9,:,:]

In [ ]:
fig = plt.figure(1, figsize=(13,8))
ax = plt.subplot(projection=ccrs.PlateCarree())
plot_date = mdates.num2date(ds.mtime.values[time_index])
plt.title('Surface Temperature (100$m$) \n CaliROMS on {}'.format(plot_date))

max_v = np.nanmax(var)
min_v = np.nanmin(var)
kw = dict(levels=np.linspace(min_v,max_v,101), 
          cmap=cmo.thermal, 
          transform=ccrs.PlateCarree())

plt.contourf(ds.lon_rho.values, ds.lat_rho.values, var, **kw)
cb = plt.colorbar(pad=0.01)
cb.set_label('Temperature ($^{\circ}$C)',rotation=270, labelpad=25, fontsize=12)

### High resolution coastline is set by resolution='10m'
ax.coastlines(resolution='50m', color='black', linewidth=1)
ax.add_feature(cartopy.feature.LAND, zorder=0)
ax.add_feature(cartopy.feature.OCEAN, zorder=0, color='gray', alpha=0.1)

gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=1, color='gray', alpha=0.2, linestyle='--')
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
gl.xlabels_top = False
gl.ylabels_right = False
gl.xlabel_style = {'size': 14}
gl.ylabel_style = {'size': 14}

plt.show()